In [1]:
import socket
import mediapipe as mp
import cv2
import pickle
import threading
import asyncio
import pandas as pd
from dollarpy import Point
from bleak import BleakScanner
from openpyxl import load_workbook

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\Users\Lenovo\Videos\Smart_Shopping-main\bluetooth_devices.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['MAC Address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

async def scan_for_devices():
    devices = await BleakScanner.discover()
    new_entries = []

    for device in devices:
        mac_address = device.address
        device_name = device.name
        print(f"Device: {device_name}, MAC Address: {mac_address}")
        
        # Check if MAC address is already registered
        if mac_address in registered_macs:
            msg_pred = f"logged in\n".encode("utf-8")
        else:
            # Register new MAC address and update the registered set
            msg_pred = f"registered\n".encode("utf-8")
            registered_macs.add(mac_address)
            new_entries.append({'MAC Address': mac_address, 'Device Name': device_name})  # Add to new entries
        
        # Send message
        conn.send(msg_pred)

    # Append new entries directly to the Excel file if there are any
    if new_entries:
        try:
            workbook = load_workbook(excel_path)
            sheet = workbook.active
            
            # Find the next empty row and append new entries
            for entry in new_entries:
                sheet.append([entry['MAC Address'], entry['Device Name']])
            
            # Save the workbook
            workbook.save(excel_path)
            print("Updated Excel with new registered devices.")
        
        except Exception as e:
            print("Error updating Excel file:", e)

def scan_in_background():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(scan_for_devices())

scanner_thread = threading.Thread(target=scan_in_background)

scanner_thread.start()

scanner_thread.join()

conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()


Device connected
Device: net, MAC Address: D8:34:1C:FA:05:97
Device: None, MAC Address: 31:2A:01:04:46:9E
Updated Excel with new registered devices.


In [1]:
!pip install deepface

In [8]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
import time
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
from bleak import BleakScanner

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bluetooth_devices.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['MAC Address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")


# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)  # Send data to the client
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break


def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

async def scan_for_devices():
    devices = await BleakScanner.discover()
    new_entries = []

    for device in devices:
        mac_address = device.address
        device_name = device.name
        print(f"Device: {device_name}, MAC Address: {mac_address}")
        
        # Check if MAC address is already registered
        if mac_address in registered_macs:
            msg_pred = f"logged in\n".encode("utf-8")
        else:
            # Register new MAC address and update the registered set
            msg_pred = f"registered\n".encode("utf-8")
            registered_macs.add(mac_address)
            new_entries.append({'MAC Address': mac_address, 'Device Name': device_name})  # Add to new entries
        
        # Send message
        conn.send(msg_pred)

    # Append new entries directly to the Excel file if there are any
    if new_entries:
        try:
            workbook = load_workbook(excel_path)
            sheet = workbook.active
            
            # Find the next empty row and append new entries
            for entry in new_entries:
                sheet.append([entry['MAC Address'], entry['Device Name']])
            
            # Save the workbook
            workbook.save(excel_path)
            print("Updated Excel with new registered devices.")
        
        except Exception as e:
            print("Error updating Excel file:", e)

def scan_in_background():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(scan_for_devices())

# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)

camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()

camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()

cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()

Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded
Hand landmarks: [(0.8045686483383179, 0.9413448572158813), stroke 1, (0.8035512566566467, 0.8706986904144287), stroke 1, (0.7744861245155334, 0.8091647028923035), stroke 1, (0.7380509376525879, 0.7768495678901672), stroke 1, (0.7061597108840942, 0.7674751281738281), stroke 1]
Predictions: (None, 0)
Detected gesture: None
Hand landmarks: [(0.8112119436264038, 0.9446784853935242), stroke 1, (0.8055464029312134, 0.8694310188293457), stroke 1, (0.7815361618995667, 0.8056544065475464), stroke 1, (0.7497886419296265, 0.7688838839530945), stroke 1, (0.721277117729187, 0.7541846036911011), stroke 1]
Predictions: (None, 0)
Detected gesture: None
Hand landmarks: [(0.8120874762535095, 0.9732627272605896), stroke 1, (0.812046468257904, 0.892183244228363), stroke 1, (0.7883259654045105, 0.8305051922798157), stroke 1, (0.7597299218177795, 0.7956746220588684), stroke 1, (0.7317475080490112, 0.782380878925323

In [7]:
!pip install bleak


In [3]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 2.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 2.3 MB/s eta 0:00:00


In [4]:
!pip install --upgrade deepface


In [ ]:
q

Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded
Starting continuous scanning for Bluetooth devices (each scan lasts 5 seconds)...
Scanning for Bluetooth devices...
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.

No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
Detected device: Youssef’s AirPods Pro - Find My, MAC Address

In [1]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import bluetooth
import asyncio
from openpyxl import load_workbook  # For Excel file manipulation

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)  # Send data to the client
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Bluetooth function using asyncio

async def continuous_bluetooth_login(scan_duration=3, cycles=1):
    print(f"Starting continuous scanning for Bluetooth devices (each scan lasts {scan_duration} seconds)...")

    try:
        while True:  # Continuous scanning loop
            print("Scanning for Bluetooth devices...")
            all_devices = []  # List to collect all devices detected in the current cycle

            # Perform multiple scan cycles per round
            for _ in range(cycles):
                nearby_devices = bluetooth.discover_devices(duration=scan_duration, lookup_names=True, device_id=-1, flush_cache=True)

                for addr, name in nearby_devices:
                    all_devices.append((addr, name))

            # Remove duplicates based on MAC address
            unique_devices = {device[0]: device[1] for device in all_devices}.items()

            if not unique_devices:
                print("No devices found during this scan.")
            else:
                for mac_address, device_name in unique_devices:
                    print(f"Detected device: {device_name}, MAC Address: {mac_address}")

                    if mac_address in registered_macs:
                        user_data = mac_df.loc[mac_df["mac_address"] == mac_address]
                        user = user_data.iloc[0]  # Get the matched user profile
                        print(f"Login successful for user: {user['Name']}, Role: {user['role']}")
                    else:
                        print(f"Unregistered device detected: {mac_address} (Name: {device_name}).")

            await asyncio.sleep(5)  # Pause before the next scan cycle
    except Exception as e:
        print(f"An error occurred during Bluetooth scanning: {e}")

def scan_in_background():
    try:
        asyncio.run(continuous_bluetooth_login(scan_duration=5))
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            asyncio.ensure_future(continuous_bluetooth_login(scan_duration=5))


# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)
camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()
camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()
cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()


Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded
Starting continuous scanning for Bluetooth devices (each scan lasts 5 seconds)...
Scanning for Bluetooth devices...
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
Detected device: Youssef’s AirPods Pro - Find My, MAC Address: 60:FD:A6:12:5A:4A
Unregistered device detected: 60:FD:A6:12:5A:4A (Name: Youssef’s AirPods Pro - Find My).
Detected device: 010, MAC Address: 41:42:F2:39:5C:26
Unregistered device detected: 41:42:F2:39:5C:26 (Name: 010).
Detected device: P9, MAC Address: 62:DC:8D:2B:C2:80
Login successful for user: P9, Role: customer
Detected device: [TV] Samsung 7 Series (55)

: 

In [10]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
import time
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import os
from bleak import BleakScanner
from openpyxl import load_workbook

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

# Function to send data to the client (C#)
def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

# Function to capture frames from the camera
def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

# Function to capture gestures from the hand using MediaPipe
def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)  # Send data to the client
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to detect faces and emotions
def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to detect objects using YOLO
def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to scan for Bluetooth devices and send data
async def scan_for_devices():
    devices = await BleakScanner.discover()
    new_entries = []

    for device in devices:
        mac_address = device.address
        print(f"Device: {device.name}, MAC Address: {mac_address}")
        
        # Check if the MAC address exists in the Excel sheet
        device_data = mac_df[mac_df['mac_address'] == mac_address]

        if not device_data.empty:
            # Found the device in the Excel sheet, send the data
            name = device_data.iloc[0]['Name']
            role = device_data.iloc[0]['role']
            image_path = device_data.iloc[0]['image']

            # Send Name
            send_data(name.encode("utf-8"))

            # Send Role
            send_data(role.encode("utf-8"))

            # Send Image data as raw binary (if exists)
            if os.path.exists(image_path):
                with open(image_path, "rb") as img_file:
                    image_data = img_file.read()
                    send_data(image_data)  # Send image data as raw binary

            print(f"Sent data for {mac_address}: {name}, {role}")
        else:
            print(f"Device with MAC address {mac_address} not found in Excel.")

# Function to run the scanning in background
def scan_in_background():
    import asyncio
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(scan_for_devices())

# Start scanning for devices in a separate thread
scanner_thread = threading.Thread(target=scan_in_background)
scanner_thread.start()

# Start other tasks (capturing frame, detecting gestures, etc.)
camera_thread = threading.Thread(target=capture_frame)
gesture_thread = threading.Thread(target=capture_gestures)
face_thread = threading.Thread(target=detect_face_and_emotions)
object_thread = threading.Thread(target=detect_objects)

camera_thread.start()
gesture_thread.start()
face_thread.start()
object_thread.start()

# Wait for all threads to complete
camera_thread.join()
gesture_thread.join()
face_thread.join()
object_thread.join()
scanner_thread.join()
# Cleanup
shutdown_flag = True
conn.close()
soc.close()
cap.release()
cv2.destroyAllWindows()


Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.

No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
0: 416x416 (no detections), 359.7ms
Speed: 6.8ms preprocess, 359.7ms inference, 2.2ms postprocess per image at shape (1, 3, 416, 416)
No hand landmarks detected.

No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
0: 416x416 (no detections), 897.8ms
Speed: 4.0ms preprocess, 897.8ms i

In [11]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
import time
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import os
from bleak import BleakScanner
from openpyxl import load_workbook
import asyncio  # Ensure asyncio is imported

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

# Function to send data to the client (C#)
def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

# Function to capture frames from the camera
def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

# Function to capture gestures from the hand using MediaPipe
def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)  # Send data to the client
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to detect faces and emotions
def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to detect objects using YOLO
def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to scan for Bluetooth devices and send data
async def scan_for_devices():
    devices = await BleakScanner.discover()
    new_entries = []

    for device in devices:
        mac_address = device.address
        print(f"Device: {device.name}, MAC Address: {mac_address}")
        
        # Check if the MAC address exists in the Excel sheet
        device_data = mac_df[mac_df['mac_address'] == mac_address]

        if not device_data.empty:
            # Found the device in the Excel sheet, send the data
            name = device_data.iloc[0]['Name']
            role = device_data.iloc[0]['role']
            image_path = device_data.iloc[0]['image']

            # Send Name
            send_data(name.encode("utf-8"))

            # Send Role
            send_data(role.encode("utf-8"))

            # Send Image data as raw binary (if exists)
            if os.path.exists(image_path):
                with open(image_path, "rb") as img_file:
                    image_data = img_file.read()
                    send_data(image_data)  # Send image data as raw binary

            print(f"Sent data for {mac_address}: {name}, {role}")
        else:
            print(f"Device with MAC address {mac_address} not found in Excel.")

# Main entry point for starting scanning and other tasks
def main():
    # Start scanning for devices (async)
    asyncio.run(scan_for_devices())

    # Start other tasks (capturing frame, detecting gestures, etc.)
    camera_thread = threading.Thread(target=capture_frame)
    gesture_thread = threading.Thread(target=capture_gestures)
    face_thread = threading.Thread(target=detect_face_and_emotions)
    object_thread = threading.Thread(target=detect_objects)

    camera_thread.start()
    gesture_thread.start()
    face_thread.start()
    object_thread.start()

    # Wait for all threads to complete
    camera_thread.join()
    gesture_thread.join()
    face_thread.join()
    object_thread.join()

# Run main entry point
if __name__ == "__main__":
    main()

# Cleanup
shutdown_flag = True
conn.close()
soc.close()
cap.release()
cv2.destroyAllWindows()


Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded


RuntimeError: asyncio.run() cannot be called from a running event loop

In [14]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import os
from bleak import BleakScanner
import asyncio

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

# Function to send data to the client (C#)
def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

# Function to capture frames from the camera
def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        ret, frame = cap.read()
        if ret:
            if frame_queue.full():
                frame_queue.get()
            frame_queue.put(frame)
        else:
            print("Failed to capture frame.")
            shutdown_flag = True
            break

# Function to capture gestures from the hand using MediaPipe
def capture_gestures():
    global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)

    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    try:
                        prediction = recognizer.recognize(points)

                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            send_data(msg_pred)
                    except Exception as e:
                        print("An error occurred:", str(e))
            cv2.imshow('Gesture Recognition', annotated_image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to perform object detection using YOLO
def object_detection():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            results = model(frame)

            annotated_frame = frame.copy()

            for result in results:
                for det in result.boxes.data:
                    cls = int(det[5].item())
                    if cls in target_classes:
                        x1, y1, x2, y2 = map(int, det[:4])
                        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(annotated_frame, model.names[cls], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            cv2.imshow('Object Detection', annotated_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to perform face recognition
def face_recognition():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            faces = sfr.detect_known_faces(frame)

            for face in faces:
                top_left = (face['top_left'][0], face['top_left'][1])
                bottom_right = (face['bottom_right'][0], face['bottom_right'][1])
                cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
                cv2.putText(frame, face['name'], (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            cv2.imshow('Face Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Function to scan for Bluetooth devices and send data
async def scan_for_devices():
    devices = await BleakScanner.discover()
    for device in devices:
        mac_address = device.address
        device_data = mac_df[mac_df['mac_address'] == mac_address]
        if not device_data.empty:
            name = device_data.iloc[0]['Name']
            role = device_data.iloc[0]['role']
            image_path = device_data.iloc[0]['image']
            send_data(name.encode("utf-8"))
            send_data(role.encode("utf-8"))
            if os.path.exists(image_path):
                with open(image_path, "rb") as img_file:
                    image_data = img_file.read()
                    send_data(image_data)
            print(f"Sent data for {mac_address}: {name}, {role}")
        else:
            print(f"Device {mac_address} not found in Excel.")

# Start Bluetooth scanning immediately in the background
def start_bluetooth_scanning():
    loop = asyncio.get_event_loop()
    loop.create_task(scan_for_devices())

# Main function to control the flow of the application
def main():
    start_bluetooth_scanning()  # Start scanning first
    
    # Start camera and processing threads for object detection, gesture, and face recognition
    camera_thread = threading.Thread(target=capture_frame)
    object_thread = threading.Thread(target=object_detection)
    gesture_thread = threading.Thread(target=capture_gestures)
    face_thread = threading.Thread(target=face_recognition)
    
    # Start the threads
    camera_thread.start()
    object_thread.start()
    gesture_thread.start()
    face_thread.start()

    # Wait for the threads to finish
    camera_thread.join()
    object_thread.join()
    gesture_thread.join()
    face_thread.join()

if __name__ == "__main__":
    main()

shutdown_flag = True
conn.close()
soc.close()
cap.release()
cv2.destroyAllWindows()


Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded


Exception in thread Thread-119 (face_recognition):
Traceback (most recent call last):
  File "c:\Users\alie0\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\alie0\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\alie0\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\alie0\AppData\Local\Temp\ipykernel_28772\397655109.py", line 171, in face_recognition
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices



0: 320x416 1 vitamin, 686.7ms
Speed: 98.0ms preprocess, 686.7ms inference, 200.0ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 vitamin, 1182.0ms
Speed: 48.5ms preprocess, 1182.0ms inference, 14.9ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 vitamin, 2251.6ms
Speed: 14.5ms preprocess, 2251.6ms inference, 12.2ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 vitamin, 1344.0ms
Speed: 15.7ms preprocess, 1344.0ms inference, 8.1ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 vitamin, 1113.9ms
Speed: 16.0ms preprocess, 1113.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 vitamin, 308.6ms
Speed: 4.0ms preprocess, 308.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 vitamin, 289.0ms
Speed: 3.0ms preprocess, 289.0ms inference, 2.2ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 vitamin, 218.5ms
Speed: 2.0ms preprocess, 218.5ms inference, 3.2ms

Device 0A:91:6C:9B:29:62 not found in Excel.
Device B0:E4:5C:32:95:63 not found in Excel.
Device FC:03:9F:21:1C:3B not found in Excel.
Device 36:F5:2A:7E:71:0B not found in Excel.


In [15]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import bluetooth
import asyncio
from openpyxl import load_workbook  # For Excel file manipulation

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\Users\Lenovo\Desktop\hciii\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"c:\Users\Lenovo\Downloads\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

def capture_gestures():
    global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Show gesture recognition frame
            cv2.imshow('Gesture Recognition', annotated_image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Using SimpleFacerec for face recognition
            faces = sfr.detect_known_faces(frame)

            # For detecting emotions (using DeepFace)
            results = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            for result in results:
                dominant_emotion = result['dominant_emotion']
                cv2.putText(frame, f"Emotion: {dominant_emotion}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            for face in faces:
                top_left = (face['top_left'][0], face['top_left'][1])
                bottom_right = (face['bottom_right'][0], face['bottom_right'][1])
                cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
                cv2.putText(frame, face['name'], (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Show face recognition with emotion frame
            cv2.imshow('Face and Emotion Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            results = model(frame)

            annotated_frame = frame.copy()

            for result in results:
                for det in result.boxes.data:
                    cls = int(det[5].item())
                    if cls in target_classes:
                        x1, y1, x2, y2 = map(int, det[:4])
                        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(annotated_frame, model.names[cls], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            # Show object detection frame
            cv2.imshow('Object Detection', annotated_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Bluetooth function using asyncio
async def continuous_bluetooth_login(scan_duration=3, cycles=1):
    print(f"Starting continuous scanning for Bluetooth devices (each scan lasts {scan_duration} seconds)...")

    try:
        while True:  # Continuous scanning loop
            print("Scanning for Bluetooth devices...")
            all_devices = []  # List to collect all devices detected in the current cycle

            # Perform multiple scan cycles per round
            for _ in range(cycles):
                nearby_devices = bluetooth.discover_devices(duration=scan_duration, lookup_names=True, device_id=-1, flush_cache=True)

                for addr, name in nearby_devices:
                    all_devices.append((addr, name))

            # Remove duplicates based on MAC address
            unique_devices = {device[0]: device[1] for device in all_devices}.items()

            if not unique_devices:
                print("No devices found during this scan.")
            else:
                for mac_address, device_name in unique_devices:
                    print(f"Detected device: {device_name}, MAC Address: {mac_address}")

                    if mac_address in registered_macs:
                        user_data = mac_df.loc[mac_df["mac_address"] == mac_address]
                        user = user_data.iloc[0]  # Get the matched user profile
                        print(f"Login successful for user: {user['Name']}, Role: {user['role']}")
                    else:
                        print(f"Unregistered device detected: {mac_address} (Name: {device_name}).")

            await asyncio.sleep(5)  # Pause before the next scan cycle
    except Exception as e:
        print(f"An error occurred during Bluetooth scanning: {e}")

def scan_in_background():
    try:
        asyncio.run(continuous_bluetooth_login(scan_duration=5))
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            asyncio.ensure_future(continuous_bluetooth_login(scan_duration=5))

# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)

camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()

camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()

cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()


Error reading Excel file: [Errno 2] No such file or directory: 'C:\\Users\\Lenovo\\Desktop\\hciii\\Smart_Shopping-main\\bloutooth.xlsx'


: 

In [1]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import bluetooth
import asyncio
from openpyxl import load_workbook  # For Excel file manipulation

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

# Send data to the client with message type prefix
def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                msg_with_newline = msg + '\n'  # Append \n to the message
                conn.send(msg_with_newline.encode('utf-8'))  # Send the message with newline
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()


# Capture frames from the camera
def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

# Capture hand gestures and send detected gestures to the client
def capture_gestures():
    global shutdown_flag  
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            send_data("GESTURE", detected_gesture)
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Detect faces and emotions
def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

            send_data("FACE", f"{name} is {emotion}")

# Detect objects using YOLO
def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        send_data("OBJECT", f"Detected Object: {class_name}")

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Bluetooth function using asyncio
async def continuous_bluetooth_login(scan_duration=3, cycles=1):
    print(f"Starting continuous scanning for Bluetooth devices (each scan lasts {scan_duration} seconds)...")

    try:
        while True:  # Continuous scanning loop
            print("Scanning for Bluetooth devices...")
            all_devices = []  # List to collect all devices detected in the current cycle

            # Perform multiple scan cycles per round
            for _ in range(cycles):
                nearby_devices = bluetooth.discover_devices(duration=scan_duration, lookup_names=True, device_id=-1, flush_cache=True)

                for addr, name in nearby_devices:
                    all_devices.append((addr, name))

            # Remove duplicates based on MAC address
            unique_devices = {device[0]: device[1] for device in all_devices}.items()

            if not unique_devices:
                print("No devices found during this scan.")
            else:
                for mac_address, device_name in unique_devices:
                    print(f"Detected device: {device_name}, MAC Address: {mac_address}")

                    if mac_address in registered_macs:
                        user_data = mac_df.loc[mac_df["mac_address"] == mac_address]
                        user = user_data.iloc[0]  # Get the matched user profile
                        print(f"Login successful for user: {user['Name']}, Role: {user['role']}")
                    else:
                        print(f"Unregistered device detected: {mac_address} (Name: {device_name}).")

            await asyncio.sleep(5)  # Pause before the next scan cycle
    except Exception as e:
        print(f"An error occurred during Bluetooth scanning: {e}")

def scan_in_background():
    try:
        asyncio.run(continuous_bluetooth_login(scan_duration=5))
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            asyncio.ensure_future(continuous_bluetooth_login(scan_duration=5))

# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)
camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()
camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()
cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()



Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded
Starting continuous scanning for Bluetooth devices (each scan lasts 5 seconds)...
Scanning for Bluetooth devices...
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.

No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.


Exception in thread Thread-5 (detect_face_and_emotions):
Traceback (most recent call last):
  File "c:\Users\alie0\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\alie0\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\alie0\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\alie0\AppData\Local\Temp\ipykernel_20764\3226940254.py", line 179, in detect_face_and_emotions
TypeError: send_data() takes 1 positional argument but 2 were given


No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
Detected device: Youssef’s AirPods Pro - Find My, MAC Address: 60:FD:A6:12:5A:4A
Login successful for user: youssef, Role: customer
Detected device: 010, MAC Address: 41:42:F2:39:5C:26
Unregistered device detected: 41:42:F2:39:5C:26 (Name: 010).
Detected device: P9, MAC Address: 62:DC:8D:2B:C2:80
Login successful for user: P9, Role: customer
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand land

: 